In [ ]:
import gym
from dqn import DQNAgent
from network import CNN
from utils import LinearScheduler, ReplayMemory, preprocess, get_random_states, plot

In [ ]:
env = gym.make('Breakout-v4')

In [ ]:
final_eps = 0.1
initial_eps = 1.0
max_steps = 1000000
eps_scheduler = LinearScheduler(initial_eps, final_eps, max_steps)
target_obs_size = (1, 110, 84)
replay_memory = ReplayMemory(60000)
random_states = get_random_states(env, target_obs_size, 256, env.action_space.n)

In [ ]:
q_net = CNN(env.action_space.n)
agent = DQNAgent(env.action_space.n, replay_memory, preprocess, target_obs_size, eps_scheduler, 64)
SKIP_FRAMES = 3
NUM_EPISODES = 10000
frames_seen = 0
frames = 0
total_reward = 0.0
for i in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    eps_reward = 0.0
    reward, done = 0.0, False
    while not done:    
        action, eps = agent.policy(observation, reward, done)
        frames_seen += 1
        reward = 0.0
        for _ in range(SKIP_FRAMES+1):
            frames += 1
            observation, r, done, _ = env.step(action)
            reward += r
            if done:
                break
        eps_reward += reward
    total_reward += eps_reward
    q_val_avg = agent.evaluate(states=random_states)
    print(f"Episode {i} Epsilon = {eps:.5f} Reward = {eps_reward:.3f} Avg Reward = {total_reward/(i+1):.3f} Avg Q Val: {q_val_avg:.3f}")
    if (eps_reward) >= 100.0 or total_reward/(i+1) >= 25.0:
        break